In [7]:
#importing important libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import cv2

# Importing Deep Learning Libraries

from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop

In [8]:
train_dir = 'photos/train'
val_dir = 'photos/test'
train_datagen = ImageDataGenerator(
    rescale=1./255,    
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    horizontal_flip=True,  # randomly flip images horizontally
    vertical_flip=False, # Don't randomly flip images vertically
    )
val_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=16,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [10]:
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=16,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [11]:
emotion_model = Sequential()

In [12]:
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

In [13]:
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

In [14]:
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [15]:
emotion_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                    

In [9]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])

In [50]:
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 16,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 16)

Epoch 1/50
1794/1794 [==============================] - 147s 82ms/step - loss: 1.7773 - accuracy: 0.2730 - val_loss: 1.6747 - val_accuracy: 0.3291
Epoch 2/50
1794/1794 [==============================] - 147s 82ms/step - loss: 1.6488 - accuracy: 0.3498 - val_loss: 1.5195 - val_accuracy: 0.4176
Epoch 3/50
1794/1794 [==============================] - 153s 86ms/step - loss: 1.5492 - accuracy: 0.4002 - val_loss: 1.4048 - val_accuracy: 0.4618
Epoch 4/50
1794/1794 [==============================] - 146s 81ms/step - loss: 1.4690 - accuracy: 0.4346 - val_loss: 1.3250 - val_accuracy: 0.4958
Epoch 5/50
1794/1794 [==============================] - 146s 82ms/step - loss: 1.4005 - accuracy: 0.4654 - val_loss: 1.2709 - val_accuracy: 0.5119
Epoch 6/50
1794/1794 [==============================] - 149s 83ms/step - loss: 1.3557 - accuracy: 0.4870 - val_loss: 1.2189 - val_accuracy: 0.5333
Epoch 7/50
1794/1794 [==============================] - 148s 83ms/step - loss: 1.3117 - accuracy: 0.5009 - val_loss: 1

In [53]:
model_json = emotion_model.to_json()
with open("emotion_model_5.json", "w") as json_file:
    json_file.write(model_json)

In [54]:
emotion_model.save_weights('emotion_model_5.h5')

In [16]:
import numpy as np
from keras.models import model_from_json


emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('emotion_model_5.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("emotion_model_5.h5")
print("Loaded model from disk")

########################################
size = 300
## Angry
angry_img = cv2.imread('./emojis/angry.png')
angry_img = cv2.resize(angry_img, (size, size))

angry_img2gray = cv2.cvtColor(angry_img, cv2.COLOR_BGR2GRAY)
ret, angry = cv2.threshold(angry_img2gray, 1, 255, cv2.THRESH_BINARY)

## Disgusted
disgusted_img = cv2.imread('./emojis/disgusted.jfif')
disgusted_img = cv2.resize(disgusted_img, (size, size))

disgusted_img2gray = cv2.cvtColor(disgusted_img, cv2.COLOR_BGR2GRAY)
ret, disgusted = cv2.threshold(disgusted_img2gray, 1, 255, cv2.THRESH_BINARY)

## Fearfull
fearful_img = cv2.imread('./emojis/fearful.png')
fearful_img = cv2.resize(fearful_img, (size, size))

fearful_img2gray = cv2.cvtColor(fearful_img, cv2.COLOR_BGR2GRAY)
ret, fearful = cv2.threshold(fearful_img2gray, 1, 255, cv2.THRESH_BINARY)

## Happy
happy_img = cv2.imread('./emojis/happy.png')
happy_img = cv2.resize(happy_img, (size, size))

happy_img2gray = cv2.cvtColor(happy_img, cv2.COLOR_BGR2GRAY)
ret, happy = cv2.threshold(happy_img2gray, 1, 255, cv2.THRESH_BINARY)

## neutral
neutral_img = cv2.imread('./emojis/neutral.png')
neutral_img = cv2.resize(neutral_img, (size, size))

neutral_img2gray = cv2.cvtColor(neutral_img, cv2.COLOR_BGR2GRAY)
ret, neutral = cv2.threshold(neutral_img2gray, 1, 255, cv2.THRESH_BINARY)

## sad
sad_img = cv2.imread('./emojis/sad.png')
sad_img = cv2.resize(sad_img, (size, size))

sad_img2gray = cv2.cvtColor(sad_img, cv2.COLOR_BGR2GRAY)
ret, sad = cv2.threshold(sad_img2gray, 1, 255, cv2.THRESH_BINARY)

## surprised
surprised_img = cv2.imread('./emojis/surprised.png')
surprised_img = cv2.resize(surprised_img, (size, size))

surprised_img2gray = cv2.cvtColor(surprised_img, cv2.COLOR_BGR2GRAY)
ret, surprised = cv2.threshold(surprised_img2gray, 1, 255, cv2.THRESH_BINARY)

########################################

emotion_mask_dist = {0:angry,1:disgusted,2:fearful,3:happy,4:neutral,5:sad,6:surprised}
emotion_dist = {0:angry_img,1:disgusted_img,2:fearful_img,3:happy_img,4:neutral_img,5:sad_img,6:surprised_img}

cap = cv2.VideoCapture(0)
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier(r'haarcascades\haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (206, 235, 135), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        
        roi = frame[-size-10:-10, -size-10:-10]
        roi[np.where(emotion_mask_dist[maxindex])] = 0
        roi += emotion_dist[maxindex]
        
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Loaded model from disk
